<a href="https://colab.research.google.com/github/nipuni1313/CNN-for-image-classification/blob/main/Simple_convolutional_neural_network_to_perform_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Create train, val, and test directories
train_dir = '/content/drive/MyDrive/RealWasteDataset/train/'
val_dir = '/content/drive/MyDrive/RealWasteDataset/validation/'
test_dir = '/content/drive/MyDrive/RealWasteDataset/test/'
import os
import shutil
from sklearn.model_selection import train_test_split

# Path to your image dataset
dataset_path = '/content/drive/MyDrive/RealWaste/'

# List all categories (assumes subfolders for each category)
categories = os.listdir(dataset_path)



for dir in [train_dir, val_dir, test_dir]:
    if not os.path.exists(dir):
        os.makedirs(dir)

# Create directories for each category inside train, val, and test
for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(val_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)


In [4]:




# Split the dataset into train, validation, and test sets
for category in categories:
    category_path = os.path.join('/content/drive/MyDrive/RealWaste/', category)

    # Check if the category path exists and is a directory
    if os.path.isdir(category_path) and len(os.listdir(category_path)) > 0:
        images = os.listdir(category_path)

        # Split the data into 60% train, 20% validation, 20% test
        train_images, temp_images = train_test_split(images, test_size=0.4, random_state=42)
        val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

        # Create target category directories in train, val, and test directories if they don't exist
        os.makedirs(os.path.join(train_dir, category), exist_ok=True)
        os.makedirs(os.path.join(val_dir, category), exist_ok=True)
        os.makedirs(os.path.join(test_dir, category), exist_ok=True)

        # Move files to respective directories
        for image in train_images:
            # Check if the image is already in the target directory
            target_path = os.path.join(train_dir, category, image)
            if not os.path.exists(target_path):  # Only move if the image doesn't exist
                shutil.move(os.path.join(category_path, image), target_path)

        for image in val_images:
            # Check if the image is already in the target directory
            target_path = os.path.join(val_dir, category, image)
            if not os.path.exists(target_path):  # Only move if the image doesn't exist
                shutil.move(os.path.join(category_path, image), target_path)

        for image in test_images:
            # Check if the image is already in the target directory
            target_path = os.path.join(test_dir, category, image)
            if not os.path.exists(target_path):  # Only move if the image doesn't exist
                shutil.move(os.path.join(category_path, image), target_path)

print("Dataset split into train, validation, and test directories.")

Dataset split into train, validation, and test directories.


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize ImageDataGenerator for data augmentation (train only)
train_datagen = ImageDataGenerator(
    rescale=1./255,        # Normalize pixel values to [0, 1]
    rotation_range=20,     # Random rotations
    width_shift_range=0.2, # Random horizontal shift
    height_shift_range=0.2, # Random vertical shift
    shear_range=0.2,       # Shear transformation
    zoom_range=0.2,        # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'    # Fill empty pixels
)

# For validation and test, we just rescale the images
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data using flow_from_directory
train_data = train_datagen.flow_from_directory(
    train_dir,  # Directory with training data
    target_size=(128, 128),    # Resize images
    batch_size=32,
    class_mode='sparse'  # Set to 'sparse' to get integer labels
)

val_data = val_test_datagen.flow_from_directory(
    val_dir,  # Directory with validation data
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'  # Set to 'sparse' to get integer labels
)

test_data = val_test_datagen.flow_from_directory(
    test_dir,  # Directory with test data
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'  # Set to 'sparse' to get integer labels
)


Found 2914 images belonging to 9 classes.
Found 950 images belonging to 9 classes.
Found 958 images belonging to 9 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the CNN model
model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output
model.add(Flatten())

# Fully Connected Layer
model.add(Dense(128, activation='relu'))

# Add Dropout to reduce overfitting
model.add(Dropout(0.5))

# Output Layer with softmax activation for multi-class classification
model.add(Dense(train_data.num_classes, activation='softmax'))

# Compile the model with Adam optimizer and sparse categorical crossentropy loss
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



# Summary of the model
model.summary()


In [ ]:
import matplotlib.pyplot as plt

history = model.fit(
    train_data,
    steps_per_epoch=train_data.samples // train_data.batch_size,
    epochs=20,
    validation_data=val_data,
    validation_steps=val_data.samples // val_data.batch_size
)


# Plot training & validation loss and accuracy
plt.figure(figsize=(12, 6))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
